<a href="https://colab.research.google.com/github/chinmay5/NLP-Praktikum/blob/sentence_embedding/Data_Tokenization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This is an attempt to include the LASER multi lingual embedding


In [1]:
!git clone https://github.com/ceshine/LASER.git

fatal: destination path 'LASER' already exists and is not an empty directory.


In [2]:
%env LASER=/content/LASER
!echo $LASER

env: LASER=/content/LASER
/content/LASER


In [3]:
%cd LASER

/content/LASER


In [4]:
!bash install_models.sh

 - /content/LASER/models/bilstm.93langs.2018-12-26.pt already downloaded
 - /content/LASER/models/93langs.fcodes already downloaded
 - /content/LASER/models/93langs.fvocab already downloaded


In [5]:
!bash install_external_tools.sh

Installing external tools

automatic installation of the Japanese tokenizer mecab may be tricky
Please install it manually from https://github.com/taku910/mecab

The installation directory should be /content/LASER/tools-external/mecab



In [6]:
%cd tools-external/fastBPE/
! g++ -std=c++11 -pthread -O3 fastBPE/main.cc -IfastBPE -o fast

/content/LASER/tools-external/fastBPE


In [0]:
import fastBPE

In [8]:
%cd ../../tasks/similarity/
! ls

/content/LASER/tasks/similarity
dev  embed  README.md  wmt.sh


In [0]:
! apt install libopenblas-base libomp-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libopenblas-base is already the newest version (0.2.20+ds-4).
libopenblas-base set to manually installed.
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
Suggested packages:
  libomp-doc
The following NEW packages will be installed:
  libomp-dev libomp5
0 upgraded, 2 newly installed, 0 to remove and 8 not upgraded.
Need to get 239 kB of archives.
After this operation, 804 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libomp5 amd64 5.0.1-1 [234 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libomp-dev amd64 5.0.1-1 [5,088 B]
Fetched 239 kB in 0s (712 kB/s)
Selecting previously unselected package libomp5:amd64.
(Reading database ... 130912 files and directories currently installed.)
Preparing to unpack .../libomp5_5.0.1-1_amd64.deb ..

In [9]:
! pip install faiss

In [10]:
 % cd ./tasks/similarity
! bash wmt.sh

[Errno 2] No such file or directory: './tasks/similarity'
/content/LASER/tasks/similarity
LASER: similarity search

Processing:
 - loading encoder /content/LASER/models/bilstm.93langs.2018-12-26.pt
 - transfer encoder to GPU
 - Tokenizer: newstest2012.tok.cs exists already
 - fast BPE: newstest2012.bpe.cs exists already
 - Encoder: newstest2012.enc.cs exists already
 - embedding: ./embed/newstest2012.enc.cs 3003 examples of dim 1024
 - creating FAISS index
 - Tokenizer: newstest2012.tok.de exists already
 - fast BPE: newstest2012.bpe.de exists already
 - Encoder: newstest2012.enc.de exists already
 - embedding: ./embed/newstest2012.enc.de 3003 examples of dim 1024
 - creating FAISS index
 - Tokenizer: newstest2012.tok.en exists already
 - fast BPE: newstest2012.bpe.en exists already
 - Encoder: newstest2012.enc.en exists already
 - embedding: ./embed/newstest2012.enc.en 3003 examples of dim 1024
 - creating FAISS index
 - Tokenizer: newstest2012.tok.es exists already
 - fast BPE: newst

In [0]:
import os
import sys
LASER = os.environ['LASER']
# now include the extra files in the source
sys.path.append(LASER + '/source')
sys.path.append(LASER + '/source/lib')

In [0]:
from embed import SentenceEncoder, EncodeLoad, EncodeFile
from text_processing import Token, BPEfastApply
from indexing import IndexCreate, IndexSearchMultiple, IndexPrintConfusionMatrix

In [0]:
MODEL_PATH = '/content/LASER/models/'
sentence_encoder = SentenceEncoder(
    str(MODEL_PATH + "bilstm.93langs.2018-12-26.pt"),
    max_sentences=None,
    max_tokens=10000,
    cpu=False)

# Copy and paste the code by clicking on the generated link and press Enter

In [0]:
from google.colab import drive
drive.mount("/gdrive", force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


# Change the file name below

In [0]:
# Replace filename at the end
import pandas as pd
input_data = pd.read_csv('/gdrive/My Drive/data_46000_to_69000.csv')

In [0]:
input_data.columns

Index(['reviewText', 'Product_Category'], dtype='object')

In [0]:
input_data.head()

,reviewText,Product_Category
0,"A great cd, all gang-smoke songs the best of t...",Digital Music
1,It would be awesome if they would release this...,CDs and Vinyl
2,Got one of these mounted in my vehicle. Seems ...,Automotive
3,I got tired of using the old style staple remo...,Office Products
4,I've been using Ableton for 4 years and it nev...,Musical Instruments


In [0]:
input_data.Product_Category.describe()

count               23000
unique                 22
top       Office Products
freq                 1886
Name: Product_Category, dtype: object

In [0]:
input_data = input_data.rename(columns={'Product_Category':'label'})
input_data.head()

,reviewText,label
0,"A great cd, all gang-smoke songs the best of t...",Digital Music
1,It would be awesome if they would release this...,CDs and Vinyl
2,Got one of these mounted in my vehicle. Seems ...,Automotive
3,I got tired of using the old style staple remo...,Office Products
4,I've been using Ableton for 4 years and it nev...,Musical Instruments


In [0]:
LASER = os.environ['LASER']
FASTBPE = LASER + '/tools-external/fastBPE/fast'
MOSES_BDIR = LASER + '/tools-external/moses-tokenizer/tokenizer/'
MOSES_TOKENIZER = MOSES_BDIR + 'tokenizer.perl -q -no-escape -threads 20 -l '
MOSES_LC = MOSES_BDIR + 'lowercase.perl'
NORM_PUNC = MOSES_BDIR + 'normalize-punctuation.perl -l '
DESCAPE = MOSES_BDIR + 'deescape-special-chars.perl'
REM_NON_PRINT_CHAR = MOSES_BDIR + 'remove-non-printing-char.perl'
MECAB = LASER + '/tools-external/mecab'

In [0]:
from subprocess import check_output
def sample_token(string_val, lang='en'):
    tok = check_output(
                REM_NON_PRINT_CHAR
                + '|' + NORM_PUNC + lang
                + '|' + DESCAPE
                + '|' + MOSES_TOKENIZER + lang
                + ('| python3 -m jieba -d ' if lang == 'zh' else '')
                + ('|' + MECAB + '/bin/mecab -O wakati -b 50000 ' if lang == 'ja' else ''),
                input=string_val,
                encoding='UTF-8',
                shell=True)
    return tok.strip()

In [0]:
index = input_data.reviewText.isnull()==True
samp = index[index==True]
input_data = input_data.drop(samp.index)

In [0]:
input_data.reviewText.isnull().sum() #Value should be always zero

0

In [0]:
input_data['tokenized_review'] = input_data.reviewText.apply(sample_token)

In [0]:
input_data.head()

,reviewText,label,tokenized_review
0,"A great cd, all gang-smoke songs the best of t...",Digital Music,"A great cd , all gang-smoke songs the best of ..."
1,It would be awesome if they would release this...,CDs and Vinyl,It would be awesome if they would release this...
2,Got one of these mounted in my vehicle. Seems ...,Automotive,Got one of these mounted in my vehicle . Seems...
3,I got tired of using the old style staple remo...,Office Products,I got tired of using the old style staple remo...
4,I've been using Ableton for 4 years and it nev...,Musical Instruments,I 've been using Ableton for 4 years and it ne...


In [0]:
bpe_codes = './LASER/models/93langs.fcodes'
bpe_vocab = bpe_codes.replace('fcodes', 'fvocab')


def apply_bpe(sentence):
  sentence = sentence.replace('\\',"")
  sentence = sentence.replace('"','\\"')
  bpe = check_output(
      'echo -n "'+sentence +'" | '+ FASTBPE + ' applybpe_stream '+  bpe_codes +' '+bpe_vocab,
      encoding='UTF-8',
      shell=True)
  return bpe.strip()

In [0]:
%pwd

'/content/LASER/tasks/similarity'

In [0]:
%cd /content
tok_sentence = '"love the magnet easel ... " slow motion me "great for moving to different areas ... wish it had some sort of non skid pad on bottom though ...'
apply_bpe(tok_sentence)

/content


'"@@ love the magn@@ et ea@@ sel ... " slo@@ w motion me "@@ great for mo@@ ving to different areas ... wish it had some sort of non sk@@ id pad on bot@@ tom though ...'

In [0]:
input_data.head()

,reviewText,label,tokenized_review
0,"A great cd, all gang-smoke songs the best of t...",Digital Music,"A great cd , all gang-smoke songs the best of ..."
1,It would be awesome if they would release this...,CDs and Vinyl,It would be awesome if they would release this...
2,Got one of these mounted in my vehicle. Seems ...,Automotive,Got one of these mounted in my vehicle . Seems...
3,I got tired of using the old style staple remo...,Office Products,I got tired of using the old style staple remo...
4,I've been using Ableton for 4 years and it nev...,Musical Instruments,I 've been using Ableton for 4 years and it ne...


In [0]:
input_data['tokenized_review_bpe'] = input_data.tokenized_review.apply(apply_bpe)

# Change the file name below

In [0]:
input_data.to_csv('/gdrive/My Drive/data_46000_to_69000_tokenized.csv')

# These are the changes just to see the saving properly

In [18]:
import pandas as pd
% cd /content/
input_data = pd.read_csv('./preprocessed_output.csv', index_col=0)

/content


In [19]:
input_data.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,label,tokenized_review,tokenized_review_bpe,labels_encoded
0,A1VXOAVRGKGEAK,439893577,Angie,"[0, 0]",I like the item pricing. My granddaughter want...,5,Magnetic board,1390953600,"01 29, 2014",Toys_and_Games,I like the item pricing . My granddaughter wan...,I like the item pri@@ cing . M@@ y grand@@ dau...,21
1,A8R62G708TSCM,439893577,Candace,"[1, 1]",Love the magnet easel... great for moving to d...,4,it works pretty good for moving to different a...,1395964800,"03 28, 2014",Toys_and_Games,Love the magnet easel ... great for moving to ...,L@@ ove the magn@@ et ea@@ sel ... great for m...,21
2,A21KH420DK0ICA,439893577,capemaychristy,"[1, 1]",Both sides are magnetic. A real plus when you...,5,love this!,1359331200,"01 28, 2013",Toys_and_Games,Both sides are magnetic . A real plus when you...,B@@ ot@@ h sides are mag@@ ne@@ tic . A real p...,21
3,AR29QK6HPFYZ4,439893577,dcrm,"[0, 0]",Bought one a few years ago for my daughter and...,5,Daughters love it,1391817600,"02 8, 2014",Toys_and_Games,Bought one a few years ago for my daughter and...,B@@ ought one a few years ago for my dau@@ ght...,21
4,ACCH8EOML6FN5,439893577,DoyZ,"[1, 1]",I have a stainless steel refrigerator therefor...,4,Great to have so he can play with his alphabet...,1399248000,"05 5, 2014",Toys_and_Games,I have a stainless steel refrigerator therefor...,I have a sta@@ in@@ less ste@@ el re@@ fri@@ g...,21


In [0]:
# Because of the nature in which LASER tends to perform encoding we have to do this extra tweek
def to_list(element):
  return [element]

In [0]:
input_data['tokenized_review_bpe_list'] = input_data.tokenized_review_bpe.apply(to_list)


In [23]:
from pandas import Series

def progress_coroutine(print_on = 10):
    print ("Starting progress monitor")

    iterations = 0
    while True:
        yield
        iterations += 1
        if (iterations % print_on == 0):
            print ("{} iterations done".format(iterations))

def percentage_coroutine(to_process, print_on_percent = 0.10):
    print ("Starting progress percentage monitor")

    processed = 0
    count = 0
    print_count = to_process*print_on_percent
    while True:
        yield
        processed += 1
        count += 1
        if (count >= print_count):
            count = 0
            pct = (float(processed)/float(to_process))*100

            print ("{}% finished".format(pct))

def trace_progress(func, progress = None):
    def callf(*args, **kwargs):
        if (progress is not None):
            progress.send(None)

        return func(*args, **kwargs)

    return callf

def my_func(i):
    return i ** 2

data_series = input_data.tokenized_review_bpe_list
co1 = progress_coroutine()
co2 = percentage_coroutine(len(data_series))
input_data['sentence_embedding'] = data_series.apply(trace_progress(sentence_encoder.encode_sentences, progress = co2))

Starting progress percentage monitor
10.004349717268378% finished
20.008699434536755% finished
30.01304915180513% finished
40.01739886907351% finished
50.021748586341886% finished
60.02609830361026% finished
70.03044802087864% finished
80.03479773814702% finished
90.0391474554154% finished


In [24]:
# The value should be True
len(input_data.sentence_embedding.iloc[0]) == 1 

True

In [25]:
# We used this here: https://stackoverflow.com/questions/45846189/how-to-delete-all-columns-in-dataframe-except-certain-ones
input_data = input_data[['sentence_embedding','labels_encoded']]
input_data.head()

,sentence_embedding,labels_encoded
0,"[[0.009863429, -4.1575836e-06, 0.008913217, 0....",21
1,"[[0.039715618, -1.4197368e-05, 0.017486058, 0....",21
2,"[[0.012652903, -2.9125708e-06, 0.0009350272, 0...",21
3,"[[0.014141707, -1.0407556e-05, 0.00032491196, ...",21
4,"[[0.0115525685, 0.0019211376, 0.008262169, 0.0...",21


In [0]:
input_data.to_pickle('./pickle_data')

In [0]:
# Now hopefuly reading looks good as well
input_new = pd.read_pickle('./pickle_data')

In [37]:
input_new.head()

,sentence_embedding,labels_encoded
0,"[[0.009863429, -4.1575836e-06, 0.008913217, 0....",21
1,"[[0.039715618, -1.4197368e-05, 0.017486058, 0....",21
2,"[[0.012652903, -2.9125708e-06, 0.0009350272, 0...",21
3,"[[0.014141707, -1.0407556e-05, 0.00032491196, ...",21
4,"[[0.0115525685, 0.0019211376, 0.008262169, 0.0...",21


In [31]:
(input_new.labels_encoded != input_data.labels_encoded).sum()

0

In [48]:
import numpy as np
def check_equality(np_array1, np_array2):
  return np.array_equal(np_array1, np_array2)

def check_similarity(series1, series2):
  """
    This is really really shitty way of doing the process but let us do it anyways
  """
  if len(series1) != len(series2):
    return False
  for idx in range(len(series1)):
    if not check_equality(series1.iloc[idx], series2.iloc[idx]):
      return false
  return True
    
print(check_similarity(input_data.sentence_embedding,input_new.sentence_embedding ))

True
